<a href="https://colab.research.google.com/github/Smruthi3/END2/blob/main/Session7-Assignment/Assignment-Part1/Session7_Assigment_sentimentanalyis_session5_without_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## Importing the standford data files

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data overview
Based on Readme file given by StandordSentiment Treebank, the following way sentence and sentiment label are mapped

1. datasetSentences.txt contains sentence and corresponding index
2. sentiment_labels contains phrase id and sentiment values ranging from 0 to 1. Here notice that there is no one to mapping between datasetSentences and sentiment_labels
5. dictionary.txt contains all phrases and their IDs
6. Now, perfom  left join on datasetSentences and dictionary using sentence and phrases
7. As we have phrase id, again perform left join on previous dataset (obtained from step #6) and sentiment_labels using phrase_id and phrase id
8. Here we get a final table with sentiment values
9. Divide the sentiment values into 5 buckets [0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.6, 0.8], (0.8, 1.0] and assign a label from 0 to 4
10. Finally, split dataset into 70% train, 30% test 


In [4]:
import os 
import pandas as pd
DATA_DIR = 'drive/My Drive/END2/Session5-Assignment/stanfordSentimentTreebank/stanfordSentimentTreebank'

In [5]:
datasetSentences = pd.read_csv(os.path.join(DATA_DIR, 'datasetSentences.txt'), sep='\t')
datasetSentences.head()

,sentence_index,sentence
0,1,The Rock is destined to be the 21st Century 's...
1,2,The gorgeously elaborate continuation of `` Th...
2,3,Effective but too-tepid biopic
3,4,If you sometimes like to go to the movies to h...
4,5,"Emerges as something rare , an issue movie tha..."


In [6]:
datasetSentences.shape

(11855, 2)

In [7]:
sentiment_label = pd.read_csv(os.path.join(DATA_DIR, "sentiment_labels.txt"),sep='|')
sentiment_label.head(),sentiment_label.shape

(   phrase ids  sentiment values
 0           0           0.50000
 1           1           0.50000
 2           2           0.44444
 3           3           0.50000
 4           4           0.42708, (239232, 2))

In [8]:
dictionary = pd.read_csv(os.path.join(DATA_DIR, "dictionary.txt"), sep='|',header=None)
dictionary.columns = ['phrases', 'phrase_id']
dictionary.head(),dictionary.shape

(       phrases  phrase_id
 0            !          0
 1          ! '      22935
 2         ! ''      18235
 3       ! Alas     179257
 4  ! Brilliant      22936, (239232, 2))

In [9]:
dataset_with_phrase_id = datasetSentences.merge(dictionary, left_on='sentence', right_on ='phrases', how='left')
dataset_with_phrase_id.head(), dataset_with_phrase_id.shape

(   sentence_index  ... phrase_id
 0               1  ...  226166.0
 1               2  ...  226300.0
 2               3  ...   13995.0
 3               4  ...   14123.0
 4               5  ...   13999.0
 
 [5 rows x 4 columns], (11855, 4))

In [10]:
ads = dataset_with_phrase_id.merge(sentiment_label, left_on='phrase_id', right_on='phrase ids', how='left')

In [11]:
ads['labels'] = [(0 if 0 <=i <=0.2 else (1 if 0.2<i<=0.4 else (2 if 0.4 <i<=0.6 else (3 if 0.6 <i<=0.8 else 4)))) for i in ads['sentiment values'] ]

In [12]:
ads.head()

,sentence_index,sentence,phrases,phrase_id,phrase ids,sentiment values,labels
0,1,The Rock is destined to be the 21st Century 's...,The Rock is destined to be the 21st Century 's...,226166.0,226166.0,0.69444,3
1,2,The gorgeously elaborate continuation of `` Th...,The gorgeously elaborate continuation of `` Th...,226300.0,226300.0,0.83333,4
2,3,Effective but too-tepid biopic,Effective but too-tepid biopic,13995.0,13995.0,0.51389,2
3,4,If you sometimes like to go to the movies to h...,If you sometimes like to go to the movies to h...,14123.0,14123.0,0.73611,3
4,5,"Emerges as something rare , an issue movie tha...","Emerges as something rare , an issue movie tha...",13999.0,13999.0,0.86111,4


In [13]:
ads.shape,ads.labels.value_counts()

((11855, 7), 1    2971
 3    2966
 4    2342
 2    2144
 0    1432
 Name: labels, dtype: int64)

In [15]:
ads.labels.value_counts()/sum(ads.labels.value_counts())*100

1    25.061156
3    25.018979
4    19.755377
2    18.085196
0    12.079291
Name: labels, dtype: float64

### Defining Data Fields

In [16]:
# Import Library
import random
import torch, torchtext
from torchtext.legacy import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [17]:
Review = data.Field(sequential = True , tokenize ='spacy',batch_first = True , include_lengths =True)
Label = data.LabelField(tokenize ='spacy',is_target=True,batch_first=True,sequential=False)

In [18]:
fields = [('reviews', Review),('labels',Label)]

In [20]:
review_examples = [data.Example.fromlist([ads.sentence[i],ads.labels[i]], fields) for i in range(ads.shape[0])] 


In [21]:
review_dataset = data.Dataset(review_examples,fields)

In [24]:
(train, test) = review_dataset.split(split_ratio=[0.70, 0.30], random_state=random.seed(SEED))

In [25]:
(len(train), len(test))

(8298, 3557)

In [26]:
vars(train.examples[10])

{'labels': 0,
 'reviews': ['In',
  'other',
  'words',
  ',',
  'about',
  'as',
  'bad',
  'a',
  'film',
  'you',
  "'re",
  'likely',
  'to',
  'see',
  'all',
  'year',
  '.']}

#### Building Vocabulary

In [35]:
MAX_VOCAB_SIZE = 25_000

Review.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

Label.build_vocab(train)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.38MB/s]                           
 99%|█████████▉| 397673/400000 [00:13<00:00, 29481.43it/s]

In [38]:
print('Size of input vocab : ', len(Review.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Review .vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  17071
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 7833), (',', 7004), ('the', 5925), ('and', 4349), ('of', 4288), ('a', 4264), ('to', 2972), ('-', 2645), ("'s", 2515), ('is', 2510)]
Labels :  defaultdict(None, {3: 0, 1: 1, 4: 2, 2: 3, 0: 4})


In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [40]:
batch_size=64
train_iterator , test_iterator = data.BucketIterator.splits((train,test),batch_size=batch_size,sort_key= lambda x: len(x.reviews),
                                                             sort_within_batch=True,device=device)

In [41]:
import os, pickle
with open('drive/My Drive/END2/Session7-Assignment/Assignment-part1/tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Review.vocab.stoi, tokens)

### Defining the Model

In [42]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout, pad_idx, bidirectional=False):
        
        super().__init__()          
        
        self.bidirectional = bidirectional
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           bidirectional=bidirectional,
                           batch_first=True)

        
        # Dense layer
        self.dropout = nn.Dropout(dropout)
        scaling_factor = 2 if bidirectional else 1
        
        self.fc = nn.Linear(hidden_dim*scaling_factor, output_dim)

    def forward(self, text, text_lengths):
      
        # text = [batch size, sent_length]
        embedded = self.dropout(self.embedding(text))
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        if self.bidirectional:
          hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
          hidden = self.dropout(hidden)
          hidden = hidden[0]
          
        dense_outputs = self.fc(hidden)   
        # Final activation function softmax
        output = F.softmax(dense_outputs, dim=1)
            
        return output

In [43]:
# Define hyperparameters
size_of_vocab = len(Review.vocab)
embedding_dim = 300
num_hidden_nodes = 256
num_output_nodes = 5
num_layers = 2
dropout = 0.5
bidirectional=True
PAD_IDX = Review.vocab.stoi[Review.pad_token]


# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout, pad_idx=PAD_IDX, bidirectional=bidirectional)

In [44]:
UNK_IDX = Review.vocab.stoi[Review.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[PAD_IDX] = torch.zeros(embedding_dim)

In [45]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(17071, 300, padding_idx=1)
  (encoder): LSTM(300, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=5, bias=True)
)
The model has 7,843,609 trainable parameters


#### Model Training and Evaluation

In [46]:
import torch.optim as optim

# define optimizer and loss
# lr=0.005
# lr = 2e-4
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# define metric
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    _, predictions = torch.max(preds, 1)
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

Training Loop

In [47]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        review, review_lengths = batch.reviews   
        
        # convert to 1D tensor
        predictions = model(review, review_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        #acc = binary_accuracy(predictions, batch.labels)   
        acc = categorical_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Evaluation Loop

In [48]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            review, review_lengths = batch.reviews 
            
            # convert to 1d tensor
            predictions = model(review, review_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            # acc = binary_accuracy(predictions, batch.labels)
            acc = categorical_accuracy(predictions, batch.labels)   
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Model Training and Evaluation 

In [49]:
N_EPOCHS = 30
best_test_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    test_loss, test_acc = evaluate(model, test_iterator, criterion)
    
    
    # save the best model
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), 'drive/My Drive/END2/Session7-Assignment/Assignment-part1/saved_weights.pt')
    
    print(f'\t Epoch: {epoch} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Epoch: {epoch} | Val. Loss: {test_loss:.3f} |  Val. Acc: {test_acc*100:.2f}% \n')

	 Epoch: 0 | Train Loss: 1.575 | Train Acc: 27.84%
	 Epoch: 0 | Val. Loss: 1.541 |  Val. Acc: 33.01% 

	 Epoch: 1 | Train Loss: 1.527 | Train Acc: 35.10%
	 Epoch: 1 | Val. Loss: 1.518 |  Val. Acc: 36.37% 

	 Epoch: 2 | Train Loss: 1.492 | Train Acc: 39.29%
	 Epoch: 2 | Val. Loss: 1.519 |  Val. Acc: 36.92% 

	 Epoch: 3 | Train Loss: 1.461 | Train Acc: 42.65%
	 Epoch: 3 | Val. Loss: 1.520 |  Val. Acc: 36.49% 

	 Epoch: 4 | Train Loss: 1.431 | Train Acc: 46.01%
	 Epoch: 4 | Val. Loss: 1.518 |  Val. Acc: 36.69% 

	 Epoch: 5 | Train Loss: 1.401 | Train Acc: 49.09%
	 Epoch: 5 | Val. Loss: 1.503 |  Val. Acc: 38.19% 

	 Epoch: 6 | Train Loss: 1.384 | Train Acc: 51.09%
	 Epoch: 6 | Val. Loss: 1.505 |  Val. Acc: 38.39% 

	 Epoch: 7 | Train Loss: 1.362 | Train Acc: 53.57%
	 Epoch: 7 | Val. Loss: 1.497 |  Val. Acc: 39.22% 

	 Epoch: 8 | Train Loss: 1.349 | Train Acc: 54.82%
	 Epoch: 8 | Val. Loss: 1.502 |  Val. Acc: 38.58% 

	 Epoch: 9 | Train Loss: 1.321 | Train Acc: 57.74%
	 Epoch: 9 | Val. Loss

### Validation of the model by passing the reviews and observing it's outcome

In [75]:
#load weights and tokenizer

path = 'drive/My Drive/END2/Session7-Assignment/Assignment-part1/saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('drive/My Drive/END2/Session7-Assignment/Assignment-part1/tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

# labels: very negative, negative, neutral, positive, very positive
# defaultdict(None, {3: 0, 1: 1, 4: 2, 2: 3, 0: 4})
categories = {4: "very negative", 1:"negative", 3:"neutral", 0:'positive', 2: 'very positive'}

def classify_review(review):
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(review)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [98]:

correctly_classified_review = []
correctly_classified_act_label = []
correctly_classified_pred_label = []

miss_classified_review = []
miss_classified_act_label = []
miss_classified_pred_label = []

for n in range(0,len(test)):
 # n = random.randint(0,len(test))
  review = " ".join(vars(test.examples[n])['reviews']).replace(',',"")
  #print(f'Review : {review}')
  act_label = categories[vars(test.examples[n])['labels']]
  pred_label = classify_review(review)

  if(act_label==pred_label):
    correctly_classified_review.append(review)
    correctly_classified_act_label.append(act_label)
    correctly_classified_pred_label.append(pred_label)
  else:
    miss_classified_review.append(review)
    miss_classified_act_label.append(act_label)
    miss_classified_pred_label.append(pred_label)



## Correctly classified reviews

In [101]:
import random
for i in range(0,10):
  n=random.randint(0,len(correctly_classified_review))
  print(f'Review : {correctly_classified_review[n]}')
  print(f'Actual Sentiment : {correctly_classified_act_label[n]} | Predicted Sentiment : {correctly_classified_pred_label[n]} \n')

Review : Aloof and lacks any real raw emotion  which is fatal for a film that relies on personal relationships .
Actual Sentiment : positive | Predicted Sentiment : positive 

Review : The explosion essentially ruined -- or  rather  overpowered -- the fiction of the movie for me .
Actual Sentiment : negative | Predicted Sentiment : negative 

Review : The movie is like Scorsese 's Mean Streets redone by someone who ignored it in favor of old ` juvenile delinquent ' paperbacks with titles like Leather Warriors and Switchblade Sexpot .
Actual Sentiment : negative | Predicted Sentiment : negative 

Review : Mention ` ` Solaris '' five years from now and I ' m sure those who saw it will have an opinion to share .
Actual Sentiment : neutral | Predicted Sentiment : neutral 

Review : A sleek advert for youthful anomie that never quite equals the sum of its pretensions .
Actual Sentiment : negative | Predicted Sentiment : negative 

Review : Unexpected  and often contradictory  truths emerge 

## Miss classified reviews

In [102]:
import random
for i in range(0,10):
  n=random.randint(0,len(miss_classified_review))
  print(f'Review : {miss_classified_review[n]}')
  print(f'Actual Sentiment : {miss_classified_act_label[n]} | Predicted Sentiment : {miss_classified_pred_label[n]} \n')

Review : I ' ve never seen or heard anything quite like this film  and I recommend it for its originality alone .
Actual Sentiment : very negative | Predicted Sentiment : negative 

Review : There 's really only one good idea in this movie  but the director runs with it and presents it with an unforgettable visual panache .
Actual Sentiment : neutral | Predicted Sentiment : positive 

Review : Your taste for Jonah - A Veggie Tales Movie may well depend on your threshold for pop manifestations of the Holy Spirit .
Actual Sentiment : very positive | Predicted Sentiment : positive 

Review : Call me a cold - hearted curmudgeon for not being able to enjoy a mindless action movie  but I believe a movie can be mindless without being the peak of all things insipid .
Actual Sentiment : positive | Predicted Sentiment : negative 

Review : One of the smarter offerings the horror genre has produced in recent memory  even if it 's far tamer than advertised .
Actual Sentiment : neutral | Predicted 